In [81]:
import requests
import lxml.html as lh
import pandas as pd

In [129]:
url = 'https://www.iedcr.gov.bd/website/images/files/nCoV/?C=M;O=D'
get_new_files(url)

List saved as FilesList.txt
new files from website saved


### Methods


In [123]:
def get_files_list(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #look for links in the website
    tree = lh.fromstring(page.text)
    links = []
    for link_element in tree.xpath('//td//a')[1:]:
        links.append(link_element.get('href'))
    return links

def save_list(li):
    with open('FilesList.txt', 'w') as filehandle:
        for listitem in li:
            filehandle.write('%s\n' % listitem)
        print('List saved as FilesList.txt')

def load_list(): 
    lines = []
    with open('FilesList.txt', 'r') as filehandle:        
        lines = [line[:-1] for line in filehandle]
    return lines

def download_file(f):
    db_url = "https://www.iedcr.gov.bd/website/images/files/nCoV/"
    url = db_url + f
    response = requests.get(url)
    file_name = url.split('/')[-1:][0]
    with open(file_name, 'wb') as f:
        f.write(response.content)
        print(file_name +' saved!')

def get_new_files(url):
    links = get_files_list(url)
    #check for updates
    old = load_list()
    updates = []
    for item in links :    
        if item not in old:
            updates.append(item)
    #download new files in site
    for item in updates:
        download_file(item)
    #save current list
    save_list(links)
    print("new files from website saved")